In [ ]:
from utils.colors import label_color

# import miscellaneous modules

import os
import os.path as osp
import numpy as np
import time
import glob  
import keras
import models
# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf
from utils.anchors import guess_shapes, compute_locations
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import imutils
import time
import cv2 as cv
import keras
import models
from utils.image import read_image_bgr, preprocess_image, resize_image
from utils.visualization import draw_box, draw_caption
from utils.colors import label_color
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)
'''
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1.0
    sess = tf.Session(config=config)
    '''

get_session()
# use this environment flag to change which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = 'resnet50_pascal_37_0.6892.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet101')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

# load label to names mapping for visualization purposes
voc_classes = {
    'aeroplane': 0,
    'bicycle': 1,
    'bird': 2,
    'boat': 3,
    'bottle': 4,
    'bus': 5,
    'car': 6,
    'cat': 7,
    'chair': 8,
    'cow': 9,
    'diningtable': 10,
    'dog': 11,
    'horse': 12,
    'motorbike': 13,
    'person': 14,
    'pottedplant': 15,
    'sheep': 16,
    'sofa': 17,
    'train': 18,
    'tvmonitor': 19
}

labels_to_names = {}
for key, value in voc_classes.items():
    labels_to_names[value] = key

           
vid = cv.VideoCapture('walking.MP4')
writer = None
(W, H) = (None, None)
while True:
    
    stime = time.time()        
    (ret, frame) = vid.read()
    if ret == True:
        
        draw = frame
        frame = preprocess_image(frame)
        frame, scale = resize_image(frame)
        frame_shape = frame.shape[0:2]
        if W is None or H is None:
            (H,W) = frame.shape[0:2]
        feature_shape = guess_shapes(frame_shape)
        print(feature_shape)
        locations = compute_locations(feature_shape)
        for location in locations:
            
                
            print(location.shape)
            start = time.time()
            #frame = cv.resize(frame, (225,225)).astype('float32')
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(frame, axis=0))
            print('processing time:', time.time() - start)
        
            boxes /= scale
            label_to_location = {}
        
            for box, score, label in zip(boxes[0], scores[0], labels[0]):
                
                if score < 0.5:
                    break
                start_x = int(box[0])
                start_y = int(box[1])
            
                end_x = int(box[2])
                end_y = int(box[3])
                color = label_color(label)
            
                b = box.astype(int)
                
                draw_box(draw, b, color=color)
            
                caption = "{} {:.3f}".format(labels_to_names[label], score)
                draw_caption(draw, b, caption)
            if writer is None:
                fourcc = cv.VideoWriter_fourcc(*'MJPG')
                writer = cv.VideoWriter(filename='fcos_walking', apiPreference=cv.CAP_FFMPEG, fourcc=fourcc, fps=30, frameSize=(H,W), isColor=True)
            writer.write(draw)
            #draw = cv.resize(draw, (225,225))
            cv.resizeWindow('video_frame',(550,550))
            cv.imshow('video_frame', draw)
            key = cv.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
        
vid.release()
writer.release()
cv.destroyAllWindows()     
        
        

    

Using TensorFlow backend.
c:\users\allaye\appdata\local\programs\python\python37\fcos\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\allaye\appdata\local\programs\python\python37\fcos\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\allaye\appdata\local\programs\python\python37\fcos\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11], dtype=int32)]
(15698, 2)
processing time: 17.449302196502686
(3948, 2)
processing time: 1.3713047504425049
(1008, 2)
processing time: 1.39127779006958
(252, 2)
processing time: 1.3922741413116455
(66, 2)
processing time: 1.3952620029449463
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11], dtype=int32)]
(15698, 2)
processing time: 1.398256778717041
(3948, 2)
processing time: 1.3832976818084717
(1008, 2)
processing time: 1.3783094882965088
(252, 2)
processing time: 1.3842945098876953
(66, 2)
processing time: 1.3793089389801025
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11],

processing time: 1.4441351890563965
(1008, 2)
processing time: 1.4062368869781494
(252, 2)
processing time: 1.3972609043121338
(66, 2)
processing time: 1.3932709693908691
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11], dtype=int32)]
(15698, 2)
processing time: 1.401249885559082
(3948, 2)
processing time: 1.3962626457214355
(1008, 2)
processing time: 1.3942677974700928
(252, 2)
processing time: 1.3972597122192383
(66, 2)
processing time: 1.3862888813018799
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11], dtype=int32)]
(15698, 2)
processing time: 1.4062364101409912
(3948, 2)
processing time: 1.396263837814331
(1008, 2)
processing time: 1.4002535343170166
(252, 2)
processing time: 1.3952667713165283
(66, 2)
processing time: 1.3832972049713135
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array(

processing time: 1.3773162364959717
(3948, 2)
processing time: 1.381302833557129
(1008, 2)
processing time: 1.3783094882965088
(252, 2)
processing time: 1.39127779006958
(66, 2)
processing time: 1.3952670097351074
[array([ 94, 167], dtype=int32), array([47, 84], dtype=int32), array([24, 42], dtype=int32), array([12, 21], dtype=int32), array([ 6, 11], dtype=int32)]
(15698, 2)
processing time: 1.3947222232818604
(3948, 2)
processing time: 1.3763172626495361
(1008, 2)
